# Spotify Analysis: Study of song factors and the positive correlation to streaming

## Members
- Joey Beightol
- Ryan Lucero
- Matthew Parker

### Objective:


### import Libraries

In [2]:
import pandas as pd
import numpy as np


### Load in Data

In [ ]:
# Load data from a CSV file into a DataFrame
spotifyDF = pd.read_csv('spotify_2023.csv', encoding='utf-8')

# Display the first few rows of the DataFrame
print(spotifyDF.head())